In [1]:
#78. GPU上での学習
#問題77のコードを改変し，GPU上で学習を実行せよ．
from torch import nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np
import time

# シンプルな単層パーセプトロンネットワークを定義
class SingleLayerPerceptronNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        # 全結合層を定義
        self.fc = nn.Linear(input_size, output_size, bias=False)
        # 重みの初期化
        nn.init.normal_(self.fc.weight, 0.0, 1.0)

    def forward(self, x):
        x = self.fc(x)
        return x

# トレーニングデータをロード
X_train = torch.load("X_train.pt")

# モデルを初期化
model = SingleLayerPerceptronNetwork(300, 4)

# モデルの予測値をソフトマックス関数を適用して計算
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
Y_hat = torch.softmax(model.forward(X_train[:4]), dim=-1)

# ニュースデータセットを定義するクラス
class NewsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [self.x[idx], self.y[idx]]

# モデルの精度を計算する関数
def calc_acc(model, loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()

        return correct / total

# モデルの損失と精度を計算する関数（デバイス対応）
def calc_loss_acc(model, criterion, loader, device):
    model.eval()
    loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)  # デバイスに移動
            labels = labels.to(device)  # デバイスに移動
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()

        return loss / len(loader), correct / total

# トレーニング、バリデーション、テストデータをロード
X_train = torch.load("X_train.pt")
X_valid = torch.load("X_valid.pt")
X_test = torch.load("X_test.pt")
y_train = torch.load("y_train.pt")
y_valid = torch.load("y_valid.pt")
y_test = torch.load("y_test.pt")

# データセットを作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# データローダーを作成
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

# 損失関数を定義
criterion = nn.CrossEntropyLoss()

# 損失を計算してバックプロパゲーションを行う
l_1 = criterion(model(X_train[:1]), y_train[:1])
model.zero_grad()
l_1.backward()

l = criterion(model(X_train[:4]), y_train[:4])
model.zero_grad()
l.backward()

# モデルをトレーニングする関数
def train_model(dataset_train, dataset_valid, batch_size, model, criterion, num_epochs, device=None):
    model.to(device)  # モデルをデバイスに移動

    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)

    log_train = []
    log_valid = []

    for epoch in range(num_epochs):
        start_time = time.time()

        model.train()
        loss_train = 0.0
        for inputs, labels in dataloader_train:
            optimizer.zero_grad()

            inputs = inputs.to(device)  # 入力をデバイスに移動
            labels = labels.to(device)  # ラベルをデバイスに移動
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        end_time = time.time()

        # トレーニングデータとバリデーションデータでの損失と精度を計算
        loss_train, acc_train = calc_loss_acc(model, criterion, dataloader_train, device)
        loss_valid, acc_valid = calc_loss_acc(model, criterion, dataloader_valid, device)
        log_train.append([loss_train, acc_train])
        log_valid.append([loss_valid, acc_valid])

        # モデルのチェックポイントを保存
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

        print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}, train_time: {(end_time - start_time):.4f}sec')

# モデルとその他のパラメータを定義
model = SingleLayerPerceptronNetwork(300, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
num_epochs = 1

# CUDAが利用可能かどうかをチェックし、デバイスを指定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# バッチサイズごとにモデルをトレーニング
for batch_size in [2 ** i for i in range(12)]:
    print(f"batch_size : {batch_size}")
    train_model(dataset_train, dataset_valid, batch_size, model, criterion, num_epochs, device)


SyntaxError: invalid character '，' (U+FF0C) (<ipython-input-1-8b9d4ca7f488>, line 3)